In [52]:
import pandas as pd

def was_channel_jammed(channel, eventDf):
    filtered_df = eventDf[(eventDf['shortChannelId_outgoing'] == channel) | 
                          (eventDf['shortChannelId_incoming'] == channel)]
    slot_count = 0
    time_slot_list = []
    for index, row in filtered_df.iterrows():
        if row['eventType'] == 'add':
            slot_count += 1
            time_slot_list.append({'time': row['eventTimeNs'], 'taken_slots': slot_count})
        elif row['eventType'] == 'resolve':
            slot_count -= 1
            time_slot_list.append({'time': row['eventTimeNs'], 'taken_slots': slot_count})
    result_df = pd.DataFrame(time_slot_list)
    #result_df = result_df.sort_values(by='time').reset_index(drop=True)
    return result_df

def process_all_channels(pair_schedule_df):
    unique_channels = pd.unique(
        pair_schedule_df[['shortChannelId_incoming', 'shortChannelId_outgoing']].values.ravel('K')
    )
    channel_results = {}
    for channel in unique_channels:
        result_df = was_channel_jammed(channel, pair_schedule_df)
        channel_results[channel] = result_df
    return channel_results

def find_channels_with_high_slots(all_channel_results, threshold=50):
    channels_with_high_slots = []
    for channel, df in all_channel_results.items():
        if (df['taken_slots'] > threshold).any():
            channels_with_high_slots.append(channel)
    return channels_with_high_slots



In [74]:
import pandas as pd
import json

def create_pair_schedule_df(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    
    forwards = data.get('forwards', [])
    
    # Extract relevant fields
    rows = []
    for entry in forwards:
        if isinstance(entry, dict):
            rows.append({
                'eventTimeNs': int(entry['addTimeNs']),
                'eventType': 'add',
                'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
            })
            if 'resolveTimeNs' in entry:
                rows.append({
                    'eventTimeNs': int(entry['resolveTimeNs']),
                    'eventType': 'resolve',
                    'shortChannelId_outgoing': int(entry['outgoingCircuit']['shortChannelId']),
                    'shortChannelId_incoming': int(entry['incomingCircuit']['shortChannelId']),
                })
        else:
            print(f"Expected a dictionary but got {type(entry)}.")
    
    # Create DataFrame
    pair_schedule_df = pd.DataFrame(rows)
    pair_schedule_df = pair_schedule_df.sort_values(by='eventTimeNs').reset_index(drop=True)
    print(pair_schedule_df.head())
    return pair_schedule_df

# Example usage
json_file_path = 'jam by channel files/forwarding_history.json'
pair_schedule_df = create_pair_schedule_df(json_file_path)

# Now you can process the DataFrame as required
all_channel_results = process_all_channels(pair_schedule_df)
channels_with_high_slots = find_channels_with_high_slots(all_channel_results, 2)

print("Channels with taken slots above x:", channels_with_high_slots)

           eventTimeNs eventType  shortChannelId_outgoing   
0  1720360210296364736       add          362838837231616  \
1  1720360212555496736   resolve          362838837231616   
2  1720360270393194736       add          362838837231616   
3  1720360276184104736   resolve          362838837231616   
4  1720360321941107736       add          407918813970432   

   shortChannelId_incoming  
0          409018325598208  
1          409018325598208  
2          393625162809344  
3          393625162809344  
4          409018325598208  
Channels with taken slots above x: [409018325598208, 362838837231616, 374933465137152, 545357767442432, 551954837209088]


In [54]:
chan_df = was_channel_jammed(374933465137152, pair_schedule_df)

In [58]:
chan_df[chan_df['taken_slots']>320]

,time,taken_slots
3244,1720534625904518337,321
3245,1720534625918099784,322
3246,1720534626216015614,323
3247,1720534626226661924,324
3248,1720534626237376872,325
3249,1720534626254419446,326
3250,1720534626262446494,327
3251,1720534626270358642,326
3252,1720534626284205201,325
3253,1720534626302490166,324


In [50]:
chan_df['taken_slots'].max()

327